In [ ]:
#Importamos las bibliotecas necesarias
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import sys

In [4]:
#Realizamos la request
url_historiales = 'https://www.promiedos.com.ar/historiales'
req =requests.get(url_historiales)

In [5]:
#Obtengo la url de cada equipos
clubs = {}
status_code = req.status_code
if status_code == 200:
    #Creo un objeto 'BeautifulSoup'
    html = BeautifulSoup(req.text, 'html.parser')
    clubs_all_divs = html.find_all('div', id = 'clubhist')

for club in clubs_all_divs:
    clubs[club.text] = 'https://www.promiedos.com.ar/' + str(club.a['href'])
    # clubs[club.text] = 'https://www.promiedos.com.ar/' + str(club.a['href']+'&era=amateur')
    # clubs[club.text] = 'https://www.promiedos.com.ar/' + str(club.a['href']+'&era=profesional')

In [6]:
#Obtengo el historial de cada equipos
# Crear una lista vacía para almacenar los datos
data = []
df = pd.DataFrame(columns=['club_1', 'club_2', 'dif', 'pj', 'pg', 'pe', 'pp'])
for club, url in clubs.items():
        sys.stdout.write(f"Screapeando a {club}                                               \r")
        sys.stdout.flush()
        club_req = requests.get(str(url))


        if club_req.status_code == 200:
            club_html = BeautifulSoup(club_req.text, 'html.parser')
            club_historial = club_html.find('table', id = 'historial')
            club_all_tr = club_historial.find_all('tr')
            for tr in club_all_tr[1:]:
                td = tr.findAll('td')
                club_1 = club
                club_2 = td[0].text[3:]
                dif = (td[1].text)
                pj = (td[2].text)
                pg = (td[3].text)
                pe = (td[4].text)
                pp = (td[5].text)
                data.append({'club_1': club_1, 
                            'club_2': club_2, 
                            'dif': dif, #Diferencia entre el club 1 y club 2
                            'pj': pj,   #Partidos jugados
                            'pg': pg,   #Partidos ganados por el club 1 al club 2
                            'pe': pe,   #Partidos empatados
                            'pp': pp})  #Partidos perdidos por el club 1 contra el club 2

                
        else:
            print(club_req.status_code)

In [7]:
df = pd.DataFrame(data)
df.to_csv('primera_profesional.csv', index = False)

In [8]:
df

,club_1,club_2,dif,pj,pg,pe,pp
0,Aldosivi,San Lorenzo,-3,10,3,1,6
1,Aldosivi,Union,-4,10,1,4,5
2,Aldosivi,Racing Club,-4,9,2,1,6
3,Aldosivi,Boca Juniors,-4,9,2,1,6
4,Aldosivi,Huracan,-4,9,2,1,6
...,...,...,...,...,...,...,...
3868,Velez,Juv. Antoniana,+1,1,1,0,0
3869,Velez,Crucero (M),+1,1,1,0,0
3870,Velez,Bartolome Mitre (M),+1,1,1,0,0
3871,Velez,Ind Rivadavia,+1,1,1,0,0
